In [1]:
%run base.ipynb
import gym
from abides_gym_market_making_environment import *

In [2]:
# register market making env for gym use 
from gym.envs.registration import register

register(
    id="market-making-v0",
    entry_point=SubGymMarketsMarketMakingEnv_v0,
)

In [3]:
# create the environment
env = gym.make(
        "market-making-v0",
        background_config="rmsc04"
    )
#set the general seed
env.seed(1)

[1]

In [4]:
import random
state = env.reset()
done = False
counter = 0

for i in range(100):
#while not done:
    action = 0 #random.randint(0,9)
    state, reward, done, info = env.step(action)
    counter += 1
    if counter % 100 == 0:
        print("counter: {} \n state: {} \n reward: {}".format(
            counter, state, reward
        ))






no orders executed
limit sell reward: 150.0
no orders executed
no orders executed
limit buy reward: -15.0
no orders executed
no orders executed
no orders executed
no orders executed
no orders executed
no orders executed
no orders executed
no orders executed
limit buy reward: -4.5
limit buy reward: -4.5
no orders executed
limit buy reward: 211.5
limit buy reward: -1.5
no orders executed
no orders executed
no orders executed
limit buy reward: 1.5
limit sell reward: 61.5
no orders executed
no orders executed
no orders executed
no orders executed
no orders executed
no orders executed
limit buy reward: 550.0
limit sell reward: -18.0
no orders executed
limit sell reward: -4.0
no orders executed
no orders executed
no orders executed
no orders executed
limit buy reward: 55.5
limit buy reward: 40.5
limit sell reward: 37.5
limit sell reward: 112.5
no orders executed
no orders executed
limit buy reward: 150.0
no orders executed
no orders executed
no orders executed
no orders executed
no orders ex